# Principal component analysis

## Aim

The intention of this notebook is to generate the PCA analysis and plots.

Steps to generate a PCA include removing related individuals, pruning variants in linkage disequilibrium (LD), and excluding outlier samples that can suggest poor genotyping quality or distant relatedness (also restrict to individuals of homogeneous ancestry).

Pitfalls
1. Some of the PCs may capture LD structure rather than population structure (decrease in power to detect associations in these regions of high LD)
2. When projecting a new study dataset to the PCA space computed from a reference dataset: projected PCs are shrunk toward 0 in the new dataset 
3.  PC scores may capture outliers that are due to family structure, population structure or other reasons; it might be beneficial to detect and remove these individuals to maximize the population structure captured by PCA (in the case of removing a few outliers) or to restrict analyses to genetically homogeneous samples

## Input

1. bfile: the genotype array to calculate individual relationship
2. genoFile in plink format (`.bed`, `.bim` and `.fam`) - could be different from bfile in the case of using for e.g. exome data/ imputed genotype data
3. relatedness file (you can calculate relatedness using plink or king)

## Output

Form the kinship analysis
1. Kinship table

From the PCA analysis
1. values
2. vectors
3. projection (PC's)
4. loadings
5. scale
6. mahalanobis distances for outlier removal


## General workflow

1. Estimate relatedness of the individuals in the sample by using king (as implemented in Plink)
2. Select specific SNPs and samples using Plink (QC maf>1%, geno=0.1, mind=0.1 and hwe=5e-08) 
3. SNPs thining by doing LD-pruning (window=50, shift=10, r2=0.1 are the defaults) and remove related individuals prior to PCA calculation
4. First PCA run (using only unrelated individuals)
5. Calculate mahalanobis distance and create outlier removal file (criteria 6SD from the mean)
6. Re-calculate PC's without outliers
7. Project related samples 

# Command interface

In [10]:
sos run PCA.ipynb -h

usage: sos run PCA.ipynb [workflow_name | -t targets] [options] [workflow_options]
  workflow_name:        Single or combined workflows defined in this script
  targets:              One or more targets to generate
  options:              Single-hyphen sos parameters (see "sos run -h" for details)
  workflow_options:     Double-hyphen workflow-specific parameters

Workflows:
  king
  filter
  flashpca

Global Workflow Options:
  --cwd VAL (as path, required)
                        the output directory for generated files
  --bfile VAL (as path, required)
                        Path to genotype array file
  --genoFile  paths

                        Plink binary files
  --phenoFile VAL (as path, required)
                        The phenotypic file
  --job-size 1 (as int)
                        For cluster jobs, number commands to run per job
  --numThreads 1 (as int)
                        Number of threads
  --eigensoft-module '\nmodule load EIGENSOFT/7.2.1-foss-2018b\necho "Modul

## PCA analysis pipeline

In [1]:
[global]
# the output directory for generated files
parameter: cwd = path
# Path to genotype array file
parameter: bfile =path
# Plink binary files
parameter: genoFile = paths
# The phenotypic file
parameter: phenoFile = path
# For cluster jobs, number commands to run per job
parameter: job_size = 1
# Number of threads
parameter: numThreads = 1
# Number of Principal Components to output. Default is 10
parameter: k = int
# Name of the trait in the phenoFile (format FID, IID, father, mother,trait )
parameter: trait_name = str
# How to standardize X before PCA
parameter: stand = "binom2"
# Merge data (FIXME:merge step)
#parameter: merge = True
# Software container option
parameter: container_lmm = 'statisticalgenetics/lmm:1.8'

In [2]:
# Inference of relationships in the sample to remove closely related individuals
[king]
# Filter based on kinship coefficient higher than a number (e.g first degree 0.25, second degree 0.125, third degree 0.0625)
parameter: kinship = 0.0884
# bfile corresponds to the genotypic array file
input: bfile
output: f'{cwd}/{_input:bn}.kin0', related_samples = f'{cwd}/{_input:bn}.related_id'
task: trunk_workers = 1, walltime = '10h', mem = '30G', cores = numThreads, tags = f'{step_name}_{_output[0]:bn}'
bash: container=container_lmm, expand= "${ }", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout'
    plink2 \
      --bfile ${_input:n} \
      --make-king-table \
      --threads ${numThreads} \
      --out ${_output[0]:n} 
      
R:  container=container_lmm, expand= "${ }", stderr = f'{_output[1]}.stderr', stdout = f'{_output[1]}.stdout'
    library(dplyr)
    kin0 <- read.table(${_output[0]:r}, header=F)
    colnames(kin0) <- c("FID1","ID1","FID2","ID2","NSNP","HETHET","IBS0","KINSHIP")
    rel <- kin0 %>%
        filter(KINSHIP >= ${kinship})
    cat("There are", nrow(rel),"related individuals using a kinship threshold of ${kinship}\n")
    IID <- sort(unique(unlist(rel[, c("ID1", "ID2")])))
    df <- data.frame(IID)
    write.table(df,${_output[1]:r}, row.names=FALSE, col.names=FALSE)

In [ ]:
# Filter SNPs for PCA analysis and select individuals 
[filter_1]
# The path to the file that contains the list of samples to keep (format FID, IID)
parameter: keep_samples = path
# MAF filter to use
parameter: maf_filter = 0.01
# Maximum missingess per-variant
parameter: geno_filter = 0.01
# Maximum missingness per-sample
parameter: mind_filter = 0.02
# HWE filter 
parameter: hwe_filter = 5e-08
input: genoFile, group_by=1
output: f'{cwd}/cache/{_input:bn}.filtered.bed'
task: trunk_workers = 1, walltime = '10h', mem = '30G', cores = numThreads, tags = f'{step_name}_{_output:bn}'
bash: container=container_lmm, expand= "${ }", stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout'
    plink2 \
      --bfile ${_input:n} \
      ${('--maf %s' % maf_filter) if maf_filter > 0 else ''} ${('--geno %s' % geno_filter) if geno_filter > 0 else ''} ${('--hwe %s' % hwe_filter) if hwe_filter > 0 else ''} ${('--mind %s' % mind_filter) if mind_filter > 0 else ''} \
      --keep ${keep_samples}\
      --make-bed \
      --threads ${numThreads} \
      --out ${_output:n} 

In [ ]:
# Merge all the .bed files into one bed file 
[filter_2: provides=[f'{cwd}/cache/{phenoFile:bn}.filtered.merged.bed']]
input: group_by = 'all'
output: bfile_merge = f'{cwd}/cache/{phenoFile:bn}.filtered.merged.bed'
task: trunk_workers = 1, trunk_size = job_size, walltime = '48h', mem = '60G', cores = numThreads, tags = f'{step_name}_{_output:bn}'
bash: container=container_lmm, expand= "${ }", stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout'
    echo -e ${' '.join([str(x)[:-4] for x in _input[1:]])} | sed 's/ /\n/g' > ${_output:n}.merge_list
    plink \
    --bfile ${_input[0]:n} \
    --merge-list ${_output:n}.merge_list \
    --make-bed \
    --out ${_output:n} \
    --threads ${numThreads} \
    --memory 48000

In [ ]:
# LD prunning and remove related individuals (both ind of a pair)
[filter_3]
# Window size
parameter: window = 50
# Shift window every 10 snps
parameter: shift = 10
parameter: r2 = 0.1
depends: f'{cwd}/cache/{phenoFile:bn}.filtered.merged.bed'
input: named_output('related_samples')
output: f'{cwd}/cache/{phenoFile:bn}.filtered.merged.prune.in', pruned_bed = f'{cwd}/{phenoFile:bn}.filtered.merged.prune.bed'
task: trunk_workers = 1, trunk_size = job_size, walltime = '48h', mem = '60G', cores = numThreads, tags = f'{step_name}_{_output[0]:bn}'
bash: container=container_lmm, expand= "${ }", stderr = f'{_output[0]:n}.stderr', stdout = f'{_output[0]:n}.stdout'
    plink2 \
    --bfile ${_depends:n} \
    --indep-pairwise ${window} ${shift} ${r2}  \
    --out ${_output[0]:nn} \
    --threads ${numThreads} \
    --memory 48000
   
    plink2 \
    --bfile ${_depends:n} \
    --extract ${_output[0]} \
    --remove ${_input} \
    --make-bed \
    --out ${_output[1]:n} 

In [ ]:
# Run PCA analysis using flashpca
[flashpca_1]
input: f'{cwd}/{phenoFile:bn}.filtered.merged.prune.bed'
output: f'{_input:n}.pca',
        f'{_input:n}.pc1vpc2.png',
        f'{_input:n}.pc3vpc4.png',
        f'{_input:n}.pc5vpc6.png',
        f'{_input:n}.pca.mahalanobis',
        removed_outliers=f'{_input:bn}.pca.iid.no_outliers'
task: trunk_workers = 1, walltime = '10h', mem = '30G', cores = numThreads, tags = f'{step_name}_{_output[0]:bn}'
R: expand= "${ }", stderr = f'{_output[0]:n}.stderr', stdout = f'{_output[0]:n}.stdout'
    # Load required libraries
    library(dplyr)
    library(ggplot2)
    library(flashpcaR)
    library(stats)
    library(matrixStats)
    # Read the PLINK binary files
    fn <- ${_input:nr}
    # Do the PCA computation
    f <- flashpca(fn, ndim=${k}, stand="${stand}", do_loadings=TRUE, check_geno=TRUE)
    # Save the generated matrices to files
    write.table(f$values,'${_output[0]:n}.values', sep=" ", row.names=FALSE, col.names=FALSE) 
    write.table(f$vectors,'${_output[0]:n}.vectors', sep=" ", row.names=TRUE, col.names=FALSE)
    write.table(f$projection,'${_output[0]:n}.projection', sep=" ", row.names=TRUE, col.names=FALSE)
    write.table(f$loadings,'${_output[0]:n}.loadings', sep=" ", row.names=FALSE, col.names=FALSE)
    write.table(f$scale,'${_output[0]:n}.scale', sep=" ", row.names=FALSE, col.names=FALSE)
    # Use the projection file to generate plot
    pca <- read.table('${_output[0]:n}.projection', sep=" ")
    colnames(pca) <- c("ID","PC1","PC2","PC3","PC4","PC5","PC6","PC7","PC8","PC9","PC10")
    pca$IID <- sapply(strsplit(as.character(pca$ID),':'), "[", 1)
    # Read fam file with phenotypes
    pheno <- read.table(${phenoFile:r}, sep="\t", header=T )
    pca_final <-merge(pheno, pca, by="IID", all=FALSE)
    
    # Calculate mahalanobis distance
    pc <- pca_final %>%
         select("IID", "${trait_name}",starts_with("PC"))
    pc_cov <- cov(pc[,3:12])
    pc$mahal <- mahalanobis(pc[,3:12], center=FALSE, pc_cov)
    pc$p <- pchisq(pc$mahal, df=9, lower.tail=FALSE)
    # Set the cut-off to calculate outliers
    manh_dis_sq_cutoff = quantile(pc$mahal,probs = 0.997) #6 sd from the mean chi-square double sided
    cat("The cut-off for outlier removal is set to:",manh_dis_sq_cutoff,"and the number of individuals removed is:", length(which(pc$mahal >= manh_dis_sq_cutoff)),"\n")
    
    # Obtain the new sample
    new_sample = pc[(pc$mahal <= manh_dis_sq_cutoff),1]
    cat("The new sample size after outlier removal is:",length(new_sample),"\n")
  
    new_sample_df <- pc %>%
    filter((IID%in%new_sample)) %>%
    mutate(FID = IID) %>%
    select(FID, IID, ${trait_name},starts_with("PC"))
  
    new_sample_no_out <- pc %>%
    filter(IID %in% new_sample) %>%
    select("IID")
  
    # Save file with mahalanobis distance and IID for PCA recalculation
    write.table(new_sample_df,${_output[4]:r}, sep="\t", quote=FALSE, row.names=FALSE, col.names=TRUE)
    write.table(new_sample_no_out,${_output[4]:r}, sep="\t", quote=FALSE, row.names=FALSE, col.names=FALSE)
  
    # Write the PC's to a file
    write.table(pca_final,${_output[0]:r}, sep="\t", quote=FALSE, row.names=FALSE, col.names=TRUE)
  
    # Get the min and max values for x and y-axes 
    min_axis <- round(colMins(as.matrix(f$projection[sapply(f$projection, is.numeric)])),1)
    max_axis <- round(colMaxs(as.matrix(f$projection[sapply(f$projection, is.numeric)])),1)
  
    png('${_output[1]}', width = 6, height = 4, unit='in', res=300)
    ggplot(pca_final, aes(x=PC1, y=PC2)) + geom_point(aes(color=${trait_name}, shape=${trait_name}), size=2) + labs(title="PC1 vs PC2 exomed subset ${phenoFile:bn}",x="PC1", y="PC2") +  scale_y_continuous(limits=c(min_axis, max_axis))  + scale_x_continuous(limits=c(min_axis, max_axis)) + theme_classic()
    dev.off()
  
    png('${_output[2]}', width = 6, height = 4, unit='in', res=300)
    ggplot(pca_final, aes(x=PC3, y=PC4)) + geom_point(aes(color=${trait_name}, shape=${trait_name}), size=2) + labs(title="PC3 vs PC4 exomed subset ${phenoFile:bn}", x="PC3", y="PC4") +  scale_y_continuous(limits=c(min_axis, max_axis))  + scale_x_continuous(limits=c(min_axis, max_axis)) + theme_classic()
    dev.off()
    
    png('${_output[3]}', width = 6, height = 4, unit='in', res=300)
    ggplot(pca_final, aes(x=PC5, y=PC6)) + geom_point(aes(color=${trait_name}, shape=${trait_name}), size=2) + labs(title="P5 vs PC6 exomed subset ${phenoFile:bn}",x="PC5", y="PC6") +   scale_y_continuous(limits=c(min_axis, max_axis))  + scale_x_continuous(limits=c(min_axis, max_axis)) + theme_classic()
    dev.off()           

In [ ]:
# Filter out outliers from bed file and filter related individuals to project them onto PCA
[flashpca_2]
input: f'{cwd}/{phenoFile:bn}.filtered.merged.prune.bed', named_output('removed_outliers')
output: f'{cwd}/{phenoFile:bn}.filtered.merged.prune.no_outliers.bed', f'{cwd}/{phenoFile:bn}.filtered.merged.prune.related_id.bed'
task: trunk_workers = 1, trunk_size = job_size, walltime = '48h', mem = '60G', cores = numThreads, tags = f'{step_name}_{_output[0]:bn}'
bash: container=container_lmm, expand= "${ }", stderr = f'{_output[0]:n}.stderr', stdout = f'{_output[0]:n}.stdout'
    
    plink2 \
    --bfile ${_input[0]}\
    --keep ${_input[1]} \
    --make-bed \
    --out ${_output[0]:n} 
    
    plink2 \
    --bfile ${_input[0]} \
    --keep ${cwd}/${bfile:bn}.related_id\
    --make-bed \
    --out ${_output[1]:n} 

In [ ]:
# Re-do PCA without outliers
[flashpca_3]
input: f'{cwd}/{phenoFile:bn}.filtered.merged.prune.no_outliers.bed', f'{cwd}/{phenoFile:bn}.filtered.merged.prune.related_id.bed'
output: f'{_input[0]:n}.pca',
        f'{_input[0]:n}.pc1vpc2.no_out.png',
        f'{_input[0]:n}.pc3vpc4.no_out.png',
        f'{_input[0]:n}.pc5vpc6.no_out.png',
        f'{_input[0]:n}.scree_cumPVE.png',
        f'{_input[1]:n}.pca',
        f'{_input[1]:n}.projected'
task: trunk_workers = 1, walltime = '10h', mem = '30G', cores = numThreads, tags = f'{step_name}_{_output[0]:bn}'
R: expand= "${ }", stderr = f'{_output[0]:n}.stderr', stdout = f'{_output[0]:n}.stdout'
    # Load required libraries
    library(dplyr)
    library(ggplot2)
    library(flashpcaR)
    library(stats)
    library(gridExtra)
    # Read the PLINK binary files
    fn <- ${_input[0]:nr}
    # Do the PCA computation with outliers removed
    f <- flashpca(fn, ndim=${k}, stand="${stand}", do_loadings=TRUE, check_geno=TRUE)
    # Save the generated matrices to files
    write.table(f$values,'${_output[0]}.values', sep=" ", row.names=FALSE, col.names=FALSE) 
    write.table(f$vectors,'${_output[0]}.vectors', sep=" ", row.names=TRUE, col.names=FALSE)
    write.table(f$projection,'${_output[0]}.projection', sep=" ", row.names=TRUE, col.names=FALSE)
    write.table(f$loadings,'${_output[0]}.loadings', sep=" ", row.names=FALSE, col.names=FALSE)
    write.table(f$scale,'${_output[0]}.scale', sep=" ", row.names=FALSE, col.names=FALSE)
    # Use the projection file to generate plot
    pca <- read.table('${_output[0]}.projection', sep=" ")
    colnames(pca) <- c("ID","PC1","PC2","PC3","PC4","PC5","PC6","PC7","PC8","PC9","PC10")
    pca$IID <- sapply(strsplit(as.character(pca$ID),':'), "[", 1)
    # Read fam file with phenotypes
    pheno <- read.table(${phenoFile:r}, sep="\t", header=T )
    pca_final <-merge(pheno, pca, by="IID", all=FALSE)
    
    # Write the PC's to a file
    write.table(pca_final,${_output[0]:r}, sep="\t", quote=FALSE, row.names=FALSE, col.names=TRUE)
  
    # Get the min and max values for x and y-axes 
    min_axis <- round(colMins(as.matrix(f$projection[sapply(f$projection, is.numeric)])),1)
    max_axis <- round(colMaxs(as.matrix(f$projection[sapply(f$projection, is.numeric)])),1)
  
    png('${_output[1]}', width = 6, height = 4, unit='in', res=300)
    ggplot(pca_final, aes(x=PC1, y=PC2)) + geom_point(aes(color=${trait_name}, shape=${trait_name}), size=2) + labs(title="PC1 vs PC2 exomed subset ${phenoFile:bn}",x="PC1", y="PC2") +  scale_y_continuous(limits=c(min_axis, max_axis))  + scale_x_continuous(limits=c(min_axis, max_axis)) + theme_classic()
    dev.off()
  
    png('${_output[2]}', width = 6, height = 4, unit='in', res=300)
    ggplot(pca_final, aes(x=PC3, y=PC4)) + geom_point(aes(color=${trait_name}, shape=${trait_name}), size=2) + labs(title="PC3 vs PC4 exomed subset ${phenoFile:bn}", x="PC3", y="PC4") +  scale_y_continuous(limits=c(min_axis, max_axis))  + scale_x_continuous(limits=c(min_axis, max_axis)) + theme_classic()
    dev.off()
    
    png('${_output[3]}', width = 6, height = 4, unit='in', res=300)
    ggplot(pca_final, aes(x=PC5, y=PC6)) + geom_point(aes(color=${trait_name}, shape=${trait_name}), size=2) + labs(title="P5 vs PC6 exomed subset ${phenoFile:bn}",x="PC5", y="PC6") + +  scale_y_continuous(limits=c(min_axis, max_axis))  + scale_x_continuous(limits=c(min_axis, max_axis)) + theme_classic()
    dev.off() 
  
    # Create scree plot
    eigenvalues <- f$values
    PVE <- eigenvalues[,1]
    PVE <- round(PVE /sum(PVE), 2)
    PVEplot <- qplot(c(1:10), PVE) + geom_line() + xlab("Principal Component") + ylab("PVE") + ggtitle("Scree Plot") + ylim(0, 1) +scale_x_discrete(limits=factor(1:10))
    PVE_cum <- cumsum(PVE) /sum(PVE)
    cumPVEplot <- qplot(c(1:10), cumsum(PVE)) + geom_line() + xlab("Principal Component") + ylab("PVE") + ggtitle("Cumulative PVE Plot") + ylim(0, 1) + scale_x_discrete(limits=factor(1:10))
    png('${_output[4]}', width = 8, height = 4, unit='in', res=300)
    grid.arrange(PVEplot, cumPVEplot, ncol = 2)
    dev.off()
  
    # Poject related samples
    frel <- ${_input[1]:nr}
    fproject <- project(frel, loadings=f$loadings, orig_mean=f$center, orig_sd=f$scale)
    # Get the min and max values for x and y-axes 
    min_axis <- round(colMins(as.matrix(fproject$projection[sapply(fproject$projection, is.numeric)])),1)
    max_axis <- round(colMaxs(as.matrix(fproject$projection[sapply(fproject$projection, is.numeric)])),1)
  
    # Save the generated matrices to files
    write.table(f$values,'${_output[5]}.values', sep=" ", row.names=FALSE, col.names=FALSE) 
    write.table(f$vectors,'${_output[5]}.vectors', sep=" ", row.names=TRUE, col.names=FALSE)
    write.table(f$projection,'${_output[5]}.projection', sep=" ", row.names=TRUE, col.names=FALSE)
    write.table(f$loadings,'${_output[5]}.loadings', sep=" ", row.names=FALSE, col.names=FALSE)
    write.table(f$scale,'${_output[5]}.scale', sep=" ", row.names=FALSE, col.names=FALSE)
  
    # Use the projection file to generate plot for the related individuals
    pca <- read.table('${_output[5]}.projection', sep=" ")
    colnames(pca) <- c("ID","PC1","PC2","PC3","PC4","PC5","PC6","PC7","PC8","PC9","PC10")
    pca$IID <- sapply(strsplit(as.character(pca$ID),':'), "[", 1)
    # Read fam file with phenotypes
    pheno <- read.table(${phenoFile:r}, sep="\t", header=TRUE)
    pca_projection <-merge(pheno, pca, by="IID", all=FALSE)
  
    png('${_output[6]}.PC1_PC2.png', width = 6, height = 4, unit='in', res=300)
    ggplot(pca_projection, aes(x=PC1, y=PC2)) + geom_point(aes(color=${trait_name}, shape=${trait_name}), size=2) + labs(title="PC1 vs PC2 exomed subset ${phenoFile:bn}",x="PC1", y="PC2") +  scale_y_continuous(limits=c(min_axis, max_axis))  + scale_x_continuous(limits=c(min_axis, max_axis)) + theme_classic()
    dev.off()
  
    png('${_output[6]}.PC3_PC4.png', width = 6, height = 4, unit='in', res=300)
    ggplot(pca_projection, aes(x=PC3, y=PC4)) + geom_point(aes(color=${trait_name}, shape=${trait_name}), size=2) + labs(title="PC3 vs PC4 exomed subset ${phenoFile:bn}", x="PC3", y="PC4") +  scale_y_continuous(limits=c(min_axis, max_axis))  + scale_x_continuous(limits=c(min_axis, max_axis)) + theme_classic()
    dev.off()
    
    png('${_output[6]}.PC5_PC6.png', width = 6, height = 4, unit='in', res=300)
    ggplot(pca_projection, aes(x=PC5, y=PC6)) + geom_point(aes(color=${trait_name}, shape=${trait_name}), size=2) + labs(title="P5 vs PC6 exomed subset ${phenoFile:bn}",x="PC5", y="PC6") +  scale_y_continuous(limits=c(min_axis, max_axis))  + scale_x_continuous(limits=c(min_axis, max_axis)) + theme_classic()
    dev.off()